In [85]:
'''
A bunch of imports, you don't have to worry about these
'''

import numpy as np
from tqdm import tqdm
import random
import gymnasium as gym
# from gym.wrappers import Monitor
import glob
import io
import matplotlib.pyplot as plt
from IPython.display import HTML

In [86]:
'''
The environment used here is extremely similar to the openai gym ones.
At first glance it might look slightly different. 
The usual commands we use for our experiments are added to this cell to aid you
work using this environment.
'''

#Setting up the environment
env = gym.make('Taxi-v3', render_mode='ansi')
state, _ = env.reset()

# The state of the environment
print(state)

print("decoded state")
print(list(env.decode(state)))


#The number of states in the environment
print(env.observation_space.n)

#The number of actions in the environment
print(env.action_space.n)

#Take a step in the environment
print(env.step(1))
next_state, reward, done, _, _ = env.step(1)

#Render the environment
env.render()

72
decoded state
[0, 3, 3, 0]
500
6
(72, -1, False, False, {'prob': 1.0, 'action_mask': array([1, 0, 1, 1, 0, 0], dtype=int8)})


/home/adi/miniconda3/envs/rl/lib/python3.11/site-packages/gymnasium/core.py:311: UserWarning: WARN: env.decode to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.decode` for environment variables or `env.get_wrapper_attr('decode')` that will search the reminding wrappers.
  logger.warn(


'+---------+\n|\x1bR\x1b: | :\x1b \x1b:G|\n| : | : : |\n| : : : : |\n| | : | : |\n|Y| : |\x1bB\x1b: |\n+---------+\n  (North)\n'

In [87]:
total_options = 10
Q_values = np.zeros((env.observation_space.n, total_options))

# Softmax function
def softmax(Q_values, state, tau):
    q_values = Q_values[state]
    q_values = q_values / tau
    max_q = np.max(q_values)
    e = np.exp(q_values - max_q)
    dist = e / np.sum(e)
    action = np.random.choice(len(dist), p=dist)
    return action

In [88]:
# actions:  ['up', 'right', 'down', 'left', 'pickup', 'dropoff']
actions = [0, 1, 2, 3, 4, 5]

# options: ['toR', 'toG', 'toY', 'toB']
options = [6, 7, 8, 9]

def execute_option(Q_values, option, state):
    optdone = False
    optact = 0
    
    if option == 6:
        decoded_state = list(env.decode(state))
        if decoded_state[0] == 0 and decoded_state[1] == 0:
            optdone = True
        optact = softmax(Q_values=Q_values, state=state, tau=1)

    elif option == 7:
        decoded_state = list(env.decode(state))
        if decoded_state[0] == 0 and decoded_state[1] == 4:
            optdone = True
        optact = softmax(Q_values=Q_values, state=state, tau=1)

    elif option == 8:
        decoded_state = list(env.decode(state))
        if decoded_state[0] == 4 and decoded_state[1] == 0:
            optdone = True
        optact = softmax(Q_values=Q_values, state=state, tau=1)

    elif option == 9:
        decoded_state = list(env.decode(state))
        if decoded_state[0] == 4 and decoded_state[1] == 3:
            optdone = True
        optact = softmax(Q_values=Q_values, state=state, tau=1)

    return optact, optdone

In [89]:
#### SMDP Q-Learning 

# Add parameters you might need here
gamma = 0.9
alpha = 0.1
update_frequencySMDP = np.zeros((48,6))
# Iterate over 1000 episodes
for _ in tqdm(range(1000)):
    state, _ = env.reset()    
    done = False

    # While episode is not over
    while not done:
        
        # Choose action        
        action = softmax(Q_values, state, tau=1)
        # Checking if primitive action
        if action < 4:
            # Perform regular Q-Learning update for state-action pair
            next_state, reward, done, _, _ = env.step(action)
            Q_values[state, action] += alpha * (reward + gamma * np.max(Q_values[next_state]) - Q_values[state, action])
            state = next_state
            
            
        # Checking if action chosen is an option
        reward_bar = 0
        if action >= 6: # action => Away option
            count = 0
            optdone = False
            optact = 11
            current_state = state
            while (optdone == False):
                optact,optdone = execute_option(Q_values , action, state)
                action_to_take = optact
                while action_to_take >= 6:
                    action_to_take, _ = execute_option(Q_values, action, state)
                next_state, reward, done, _, _ = env.step(action_to_take)
                reward_bar = reward_bar + gamma**count*reward
                count += 1
                if optdone == True:
                    Q_values[current_state, action] += alpha * (reward_bar - Q_values[current_state, action] + gamma**count * np.max(Q_values[next_state]))
                state = next_state


  0%|          | 0/1000 [00:00<?, ?it/s]

  2%|▏         | 16/1000 [00:41<42:54,  2.62s/it] 


KeyboardInterrupt: 